<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS767/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [143]:
import random
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import math
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler 

In [144]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)
def sigmoid(x):
  output = [0] * len(x)
  for index, item in enumerate(x): 
    output[index] = 1.0 if item > 0.0 else 0.0 
  return output
def relu(x):
  output = [0] * len(x)
  for index, item in enumerate(x): 
    output[index] = item if item > 0.0 else 0.0 
  return output

In [200]:
class Layer():
  def __init__(self, name, units, input_dim):
    self.name = name
    self.w = [[random.uniform(0,1) for _ in range(input_dim)] for _ in range(units)]
    self.b = [0 for _ in range(units)]
    self.output = [bias for bias in self.b]
  def calculate_output(self, inputs, activation=sigmoid):
    self.output = [bias for bias in self.b]
    self.output += np.dot(self.w, inputs)
    self.output = activation(self.output)
    return self.output 
  def update_weights(self, error, inputs, learning_rate):
    error_out = 0
    for input_idx, input in enumerate(inputs):
      error_out += sum([learning_rate * error * weight[input_idx] for weight_idx, weight in enumerate(self.w)])
    for row_index, col in enumerate(self.w):
      for col_index, weight in enumerate(col):
        self.w[row_index][col_index] = self.w[row_index][col_index] - (learning_rate * error * inputs[col_index])
        self.b[row_index] = self.b[row_index] - (learning_rate * error)
    return error_out

In [194]:
def reset_layers():
  layer1 = Layer("Hidden1", 3, 2)
  layer2 = Layer("Output", 3, 3)
  layers = [layer1, layer2]
  return layers

In [146]:
iris = load_iris()

iris_columns = ['sepal_len', 'sepal_width', 'petal_length', 'petal_width', 'target']
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris_columns)

In [196]:
from sklearn.model_selection import train_test_split
# X = df[['sepal_len', 'sepal_width', 'petal_length', 'petal_width']]
X = df[['sepal_width', 'petal_length']]
y = df[['target']]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state = 10)


scaler_training = StandardScaler() 
scaler_training.fit(X_train)
scaled_values_training = scaler_training.transform(X_train)
scaled_values_testing = scaler_training.transform(X_test)

X_train = pd.DataFrame(scaled_values_training, columns=[*X_train.columns.values])
X_test = pd.DataFrame(scaled_values_testing, columns=[*X_test.columns.values])

# X_train['bias'] = 1
# X_test['bias'] = 1
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)


In [197]:
X_train

,sepal_width,petal_length
0,-0.398911,0.460619
1,-0.398911,0.291929
2,-0.167658,1.135382
3,-0.630163,0.741771
4,-1.323921,0.685540
...,...,...
115,-1.323921,0.685540
116,-0.398911,-0.101683
117,3.069877,-1.282518
118,0.294847,1.247843


In [148]:
import random
random.seed(34)
selected_rows = random.sample(range(0,119),   5)

In [149]:
lb = preprocessing.LabelBinarizer()
lb.fit([0,1,2])
lb.transform([0])


array([[1, 0, 0]])

In [150]:
def categorical_cross_entropy(actual, predicted):
	sum_score = 0.0
	for i in range(len(actual)):
			sum_score += actual[i] * math.log(1e-15 + predicted[i])
	mean_sum_score = (1.0 / len(actual)) * sum_score
	return -mean_sum_score

In [201]:
def train_weights_using_mini_batch_gradient_descent(X_train, y_train, num_epochs, learning_rate, layers):
  # weights for all the features 
  index = 0;
  error = 0
  mini_batch = X_train.loc[selected_rows, :]
  y_train = y_train.loc[selected_rows, :]
  while index < num_epochs:
    print("============================================")
    print("=================Iteration %d==============="%(index+1))
    print("============================================")
    for rowIndex, row in mini_batch.iterrows():
      output = row
      for layer_idx, layer in enumerate(layers):
        if layer_idx == len(layers) - 1:
          output = layer.calculate_output(output, softmax)
        else:
          output = layer.calculate_output(output, sigmoid)
      error = log_loss([y_train.loc[rowIndex,'target']], [output], labels=[0.0, 1.0, 2.0])
      print("Entropy based loss: %f"% (error))
      if error != 0:
        layer1_error = layers[1].update_weights(error, layers[0].output, learning_rate)
        layers[0].update_weights(layer1_error, row, learning_rate)
      print("New Weights of output layer", layers[1].w)
      print("New Weights of hidden layer", layers[0].w)
    # for layer_idx, layer in enumerate(layers):
    #   layer.update_weights()
    index = index + 1
  # df_plot = pd.DataFrame(df_plot, columns=['mse', 'epoch'])
  return

layers = reset_layers()
print("Creating model using training data")
train_weights_using_mini_batch_gradient_descent(X_train, y_train, 10, 0.5, layers)

Creating model using training data
=================Iteration 1===============
Entropy based loss: 1.098612
New Weights of output layer [[0.6782766303016691, 0.7977875836019059, 0.3729203719561375], [0.37241676070606033, 0.7983418887253494, 0.6052236050327029], [0.4009222186862581, 0.420449824347251, 0.7679392435730371]]
New Weights of hidden layer [[2.6168186835167146, 0.5356655828156272], [3.166451123351512, 0.3627941862695465], [2.3187479037967482, 0.9346497520338976]]
Entropy based loss: 1.098612
New Weights of output layer [[0.6782766303016691, 0.7977875836019059, 0.3729203719561375], [0.37241676070606033, 0.7983418887253494, 0.6052236050327029], [0.4009222186862581, 0.420449824347251, 0.7679392435730371]]
New Weights of hidden layer [[3.5192862747721043, -0.5266369736696637], [4.068918714606902, -0.6995083702157443], [3.221215495052138, -0.12765280445139326]]
Entropy based loss: 1.098612
New Weights of output layer [[0.6782766303016691, 0.7977875836019059, 0.3729203719561375], [0

In [202]:
for rowIndex, row in X_test.iterrows():
  # print(rowIndex)
  output = row
  for layer_idx, layer in enumerate(layers):
      if layer_idx == len(layers) - 1:
        output = layer.calculate_output(output, softmax)
      else:
        output = layer.calculate_output(output, relu)
  print(output)
  print("Predicted: %d, Actual %d"%(np.argmax(output), y_test.loc[rowIndex, 'target']))

[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 1
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 2
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 0
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 1
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 0
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 1
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 1
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 1
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 0
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 1
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 1
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 2
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 1
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 0
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 0
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 2
[0.33333333 0.33333333 0.33333333]
Predicted: 0, Actual 1
[0.33333333 0.